# Assignment 1

In [1]:
import re
import pandas as pd
import nltk

In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gener\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
df = pd.read_csv("A1_dataset.csv")

In [3]:
df.head()

,LABEL,DATE_TIME,TEXT
0,0,Fri Jun 05 14:26:50 2009,About to get threaded and scared
1,1,Thu May 14 10:13:55 2009,@awaisnaseer I like Shezan Mangooo too!!! I ha...
2,1,Fri Jun 05 21:02:20 2009,worked on my car after work. showering then go...
3,1,Sun Jun 14 22:25:52 2009,@Marama Actually we start this afternoon! I w...
4,1,Sun May 31 00:42:12 2009,@gfalcone601 Aww Gi.don't worry.we'll vote for...


### II) Text Preprocessing

In [27]:
from nltk.tokenize import TweetTokenizer

def tokenization(text):
    twt_tk = TweetTokenizer()
    return ' '.join(twt_tk.tokenize(text))

In [28]:
tokenization(df["TEXT"].iloc[0])

'About to get threaded and scared'

In [13]:
#Can use Jaccard distance or edit distance to do this

def spelling_correction(text):
    pass

In [14]:
#Here we can either do lemmatization or stemming, doing both at the same time will not be useful

from nltk.stem import WordNetLemmatizer

def stemming_lemmatization(text):
    pass

In [25]:
from nltk.tokenize import RegexpTokenizer

def remove_punctuations(text):
    return ' '.join(RegexpTokenizer(r'\w+').tokenize(text))

In [26]:
remove_punctuations(df["TEXT"].iloc[1])

'awaisnaseer I like Shezan Mangooo too I had one yesterday'

In [32]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words("english")) + r')\b\s*')
    return pattern.sub('', text)

In [33]:
remove_stopwords(df["TEXT"].iloc[1])

'@awaisnaseer I like Shezan Mangooo !!! I one yesterday '

In [51]:
def remove_whitespaces(text):
    return re.compile(r'\s*\s').sub(' ', text)

In [52]:
remove_whitespaces(df["TEXT"].iloc[1])

'@awaisnaseer I like Shezan Mangooo too!!! I had one yesterday '

In [53]:
def remove_url_html(text):
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'<(\"[^\"]*\"|\'[^\']*\'|[^\'\">])*>', '' , text)
    return text

In [54]:
remove_url_html(df["TEXT"].iloc[20])

'i hate when people have to move away from me '